<H1 style="background-color:LightGreen;">
    <center>
         Kaggle
     </center>
 </H1>

https://www.kaggle.com/competitions/spooky-author-identification/overview

Topic modeling


<H1 style="background-color:LightGreen;">
    <center>
         NLP Notes:
     </center>
 </H1>

<li> Stemming identifies the common root form of a word by removing or replacing word suffixes (e.g. “flooding” is stemmed as “flood”) 
<li> Lemmatization identifies the inflected forms of a word and returns its base form (e.g. “better” is lemmatized as “good”).

<H1 style="background-color:LightGreen;">
    <center>
         Imports
     </center>
 </H1>

In [ ]:
#-- Auto reload:
%load_ext autoreload
%autoreload 2

In [153]:
from matplotlib                          import pyplot
from matplotlib.image                    import imread
from torchvision.datasets                import ImageFolder
from torch.optim.lr_scheduler            import OneCycleLR
from sklearn.metrics                     import r2_score
from sklearn.feature_extraction.text     import TfidfVectorizer
from nltk.stem.porter                    import PorterStemmer
from nltk.corpus                         import stopwords
from nltk.tokenize                       import sent_tokenize, word_tokenize
from nltk.stem                           import WordNetLemmatizer

import numpy                  as np
import matplotlib.pyplot      as plt
import matplotlib.image       as mpimg
import torchvision.transforms as transforms
import torch.nn               as nn
import torch.nn.functional    as F
import torch.optim            as optim
import pandas                 as pd


import torchinfo
import torch
import os
import random
import matplotlib
import torchvision
import torchinfo
import gc 
import os
import time
import nltk
import string


<H1 style="background-color:LightGreen;">
    <center>
         Load Data
     </center>
 </H1>

In [25]:
TRAIN_FOLDER = "/home/amitli/Datasets/spooky-author-identification/spooky-author-identification/train.csv"
TEST_FOLDER  = "/home/amitli/Datasets/spooky-author-identification/spooky-author-identification/test.csv"

df_train = pd.read_csv(TRAIN_FOLDER)
df_train.drop(['id'], axis=1, inplace=True)
df_train.head()

,text,author
0,"This process, however, afforded me no means of...",EAP
1,It never once occurred to me that the fumbling...,HPL
2,"In his left hand was a gold snuff box, from wh...",EAP
3,How lovely is spring As we looked from Windsor...,MWS
4,"Finding nothing else, not even gold, the Super...",HPL


<H1 style="background-color:LightGreen;">
    <center>
         EDA
     </center>
 </H1>

In [26]:
df_train.groupby(by='author').count()

,text
author,
EAP,7900
HPL,5635
MWS,6044


In [145]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /home/amitli/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/amitli/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/amitli/nltk_data...
[nltk_data]   Unzipping corpora/omw-1.4.zip.


True

<H1 style="background-color:LightYellow;">
    <center>
         TOP tfidf (10 words) of each author
     </center>
 </H1>

In [138]:


def Stemmer(sentences):
    stemmer = PorterStemmer()
    
    for i in range(len(sentences)):
        words = word_tokenize(sentences[i])    
        words = [stemmer.stem(word) for word in words if word not in set(stopwords.words('english'))]
        sentences[i]=' '.join(words)
    return sentences

def Lemmmatizer(sentences):
    lemmmatizer = WordNetLemmatizer()

    for i in range(len(sentences)):
        words = word_tokenize(sentences[i])  
        words = [lemmmatizer.lemmatize(word.lower()) for word in words if word not in set(stopwords.words('english'))]
        sentences[i]=' '.join(words)   
    
    return sentences

In [152]:
vectorizer = TfidfVectorizer(stop_words="english", max_features=10)
authors    = df_train.groupby(by='author')

for author in authors:
    df_tmp = author[1].reset_index()
    res = Lemmmatizer(df_tmp['text'].values)
    res = Stemmer(res)
    X   = vectorizer.fit_transform(res)
    print(f"Author: {author[0]}, Top words: {vectorizer.get_feature_names_out()}")
   


Author: EAP, Top words: ['day' 'eye' 'great' 'howev' 'littl' 'man' 'said' 'say' 'thu' 'time']
Author: HPL, Top words: ['came' 'hous' 'like' 'look' 'man' 'night' 'old' 'saw' 'thing' 'time']
Author: MWS, Top words: ['day' 'everi' 'eye' 'feel' 'heart' 'life' 'love' 'man' 'raymond' 'time']
